## Idea Trabajo Práctico Final: Predicción de Resultado de Partidos de Tenis 
El objetivo de la red neuronal a implementar es poder predecir los resultados de cierta temporada de tenis actual considerando el historial de partidos de temporadas pasadas, como así también el historial individual entre los jugadores de cada partido. 
### Dataset de Referencia
El dataset de referencia será el historial de partidos de la Asociación de Tenistas Profesionales (ATP) brindado en su base de datos. Este no sólo incluye quien ha ganado cada partido, sino el puntaje y los porcentajes de las características relevantes de cada jugada en particular ('primeros servicios', 'segundos servicios', 'aces', 'porcentaje de devoluciones', 'doble faltas', etc.).
Por lo tanto, el set de entrenamiento de la red consistirá de un vector de características de entrada (X) representando las características de los jugadores involucrados y del partido, y un vector de salida correspondiente que determina que jugador gana. 
A continuación se realiza el parseo de datos:


In [8]:
import numpy as np

from numpy import genfromtxt
my_data_stats = genfromtxt('data/match_stats.csv', delimiter=',',dtype=None)
my_data_scores = genfromtxt('data/match_scores.csv', delimiter=',',dtype=None)


C:\Users\cufar\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  
C:\Users\cufar\Anaconda3\lib\site-packages\ipykernel_launcher.py:3: VisibleDeprecationWarning: Reading unicode strings without specifying the encoding argument is deprecated. Set the encoding, use None for the system default.
  This is separate from the ipykernel package so we can avoid doing imports until


In [39]:
#labels de matchscore

tourney_year_id = 0
tourney_order = 1
tourney_slug = 2
tourney_url_suffix = 3
tourney_round_name = 4
round_order = 5
match_order = 6
winner_name = 7
winner_player_id = 8
winner_slug = 9
loser_name = 10
loser_player_id = 11
loser_slug = 12
winner_seed = 13
loser_seed = 14
match_score_tiebreaks = 15
winner_sets_won = 16
loser_sets_won = 17
winner_games_won = 18
loser_games_won = 19
winner_tiebreaks_won = 20
loser_tiebreaks_won = 21
match_id = 22
match_stats_url_suffix = 23

#labels de matchstats
ms_tourney_order = 0
ms_match_id = 1
ms_match_stats_url_suffix = 2
ms_match_time = 3
ms_match_duration = 4
ms_winner_aces = 5
ms_winner_double_faults = 6
ms_winner_first_serves_in = 7
ms_winner_first_serves_total = 8
ms_winner_first_serve_points_won = 9
ms_winner_first_serve_points_total = 10
ms_winner_second_serve_points_won = 11
ms_winner_second_serve_points_total = 12
ms_winner_break_points_saved = 13
ms_winner_break_points_serve_total = 14
ms_winner_service_points_won = 15
ms_winner_service_points_total = 16
ms_winner_first_serve_return_won = 17
ms_winner_first_serve_return_total = 18
ms_winner_second_serve_return_won = 19
ms_winner_second_serve_return_total = 20
ms_winner_break_points_converted = 21
ms_winner_break_points_return_total = 22
ms_winner_service_games_played = 23
ms_winner_return_games_played = 24
ms_winner_return_points_won = 25
ms_winner_return_points_total = 26
ms_winner_total_points_won = 27
ms_winner_total_points_total = 28
ms_loser_aces = 29
ms_loser_double_faults = 30
ms_loser_first_serves_in = 31
ms_loser_first_serves_total = 32
ms_loser_first_serve_points_won = 33
ms_loser_first_serve_points_total = 34
ms_loser_second_serve_points_won = 35
ms_loser_second_serve_points_total = 36
ms_loser_break_points_saved = 37
ms_loser_break_points_serve_total = 38
ms_loser_service_points_won = 39
ms_loser_service_points_total = 40
ms_loser_first_serve_return_won = 41
ms_loser_first_serve_return_total = 42
ms_loser_second_serve_return_won = 43
ms_loser_second_serve_return_total = 44
ms_loser_break_points_converted = 45
ms_loser_break_points_return_total = 46
ms_loser_service_games_played = 47
ms_loser_return_games_played = 48
ms_loser_return_points_won = 49
ms_loser_return_points_total = 50
ms_loser_total_points_won = 51
ms_loser_total_points_total = 52


In [108]:
import pandas as pd
pd_my_data_scores = pd.DataFrame(data=my_data_scores)
pd_my_data_scores.columns=['tourney_year_id','tourney_order','tourney_slug','tourney_url_suffix','tourney_round_name','round_order','match_order','winner_name','winner_player_id','winner_slug','loser_name','loser_player_id','loser_slug','winner_seed','loser_seed','match_score_tiebreaks','winner_sets_won','loser_sets_won','winner_games_won','loser_games_won','winner_tiebreaks_won','loser_tiebreaks_won','match_id','match_stats_url_suffix']

Debajo se muestra un ejemplo del dataset scores obtenido:

In [110]:
pd_my_data_scores.loc[[0,1,2,3,4,5]]

,tourney_year_id,tourney_order,tourney_slug,tourney_url_suffix,tourney_round_name,round_order,match_order,winner_name,winner_player_id,winner_slug,...,loser_seed,match_score_tiebreaks,winner_sets_won,loser_sets_won,winner_games_won,loser_games_won,winner_tiebreaks_won,loser_tiebreaks_won,match_id,match_stats_url_suffix
0,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Finals',1,1,b'Nicklas Kulti',b'k181',b'nicklas-kulti',...,b'6',b'63 16 62',2,1,13,11,0,0,b'1991-7308-k181-s351',b'/en/scores/1991/7308/MS001/match-stats'
1,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Semi-Finals',2,1,b'Michael Stich',b's351',b'michael-stich',...,b'2',b'64 76(6)',2,0,13,10,1,0,b'1991-7308-s351-c243',b'/en/scores/1991/7308/MS003/match-stats'
2,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Semi-Finals',2,2,b'Nicklas Kulti',b'k181',b'nicklas-kulti',...,b'',b'75 64',2,0,13,9,0,0,b'1991-7308-k181-l206',b'/en/scores/1991/7308/MS002/match-stats'
3,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Quarter-Finals',3,1,b'Jim Courier',b'c243',b'jim-courier',...,b'Q',b'76(3) 63',2,0,13,9,1,0,b'1991-7308-c243-s367',b'/en/scores/1991/7308/MS007/match-stats'
4,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Quarter-Finals',3,2,b'Michael Stich',b's351',b'michael-stich',...,b'',b'36 63 63',2,1,15,12,0,0,b'1991-7308-s351-a031',b'/en/scores/1991/7308/MS006/match-stats'
5,b'1991-7308',1,b'adelaide',b'/en/scores/archive/adelaide/7308/1991/results',b'Quarter-Finals',3,3,b'Nicklas Kulti',b'k181',b'nicklas-kulti',...,b'',b'63 60',2,0,12,3,0,0,b'1991-7308-k181-s424',b'/en/scores/1991/7308/MS005/match-stats'


Los datos luego serán seleccionados partido a partido, a partir del dataset obtenido. Se realiza la siguiente división:

| Categoría  |  Detalle | 
|---|---|
| ID1 |  ID Jugador 1 |
| ID2 |  ID Jugador 2 |
| RANK |  Ranking ATP |
| POINTS  |  Puntos ATP | 
| FS | Porcentaje  de primeros servicios |
| W1SP | Porcentaje de primeros servicios ganados |
| W2SP | Porcentaje de segundos servicios ganados |
| WSP | Porcentaje de servicios ganados globales |
| WRP | Porcentaje de devolución ganada |
| TPW | Porcentaje de puntos totales ganados |
| TMW | Porcentaje de todos los partidos ganados |
| ACES | Promedio de aces por game |
| DF | Promedio de doble faltas por game |
| UE | Promedio de errores no forzados por game |
| WIS | Promedio de winners por game |
| BP | Porcentaje de break points ganados |
| NA | Porcentaje de puntos en la red ganados |
| A1S | Promedio de velocidad primer servicio |
| A2S | Promedio de velocidad de segundo servicio |
| FATIGUE | Fatiga por partidos jugados en 3 dias anteriores |
| RETIRED | Indica si es el primer partido luego de un retiro |
| DIRECT | Ventaja H2H entre los jugadores |

Cada partido tiene estas características que suelen indicar las diferencias entre ambos jugadores. Por lo que por ejemplo, para ATPRank, si RANK1 es el ranking del jugador 1 y RANK2 es el ranking del jugador 2, RANK = RANK1 - RANK2.
Para el entrenamiento, cada partido tendrá como salida un vector que inidque que jugador gana o que jugador pierde. Es decir, 1 si el jugador ganó o 0 si el jugador perdió.

Finalmente, el dataset será dividido en tres partes:
- Set de entrenamiento (2008 - 2014)
- Set de validación  (2015 - 2016)
- Set de prueba (2017-2018)



### Soluciones Propuestas
Se propone evaluar si utilizar una red multicapa en el entorno tensorflow para la implementación del proyecto (el cual se entrena con todo el dataset) o una red neuronal recurrente la cual posee una memoria 'selectiva' de partidos anteriores. Estas soluciones están sujetas a cambiar de acuerdo a las nuevas técnicas a aprender en la materia.

### Medición de la Calidad de la Solución Obtenida
La solución obtenida se limitará a predecir el ganador de cada partido. Esto tiene como consecuencia determinar el resultado de cada uno de los torneos a nivel ATP de la temporada, como así también el resultado del ranking al finalizar la temporada. Se contrastaran con los resultados reales, pretendiéndose como mínimo una efectividad superior al 50%.

### Proyectos Similares Implementados en Internet
El proyecto de referencia será la tesis presente en el siguiente link: https://www.doc.ic.ac.uk/teaching/distinguished-projects/2015/m.sipko.pdf el cuál plantea la predicción de partidos de tenis con diversos métodos incluyendo Machine Learning y mostrando sus resultados. Otro proyecto más breve se encuentra en el siguiente link: http://deepakn94.github.io/assets/papers/6.867.pdf. La técnica para solucionar el problema también puede referirse a predictores de otros deportes.
